# Hyperlocal Demand Forecasting & Inventory Optimization

## How to Read This Analysis

This notebook is structured to mirror real-world supply chain decision-making:

1. Validate data reliability
2. State assumptions explicitly
3. Understand demand & inventory patterns
4. Engineer decision-oriented KPIs
5. Optimize inventory policies
6. Identify operational and financial risks
7. Summarize business impact for leadership

Each section builds toward actionable decisions rather than isolated metrics.

## Business Context
This project focuses on demand-driven inventory and supply chain optimization using snapshot operational data.

Unlike time-series forecasting, this analysis optimizes:
- Inventory availability
- Stockout risk
- Overstock risk
- Cost efficiency
- Supplier and quality risks

*This mirrors real-world retail, FMCG, and D2C inventory decision-making.*

## Business Problems Addressed

The company faces the following challenges:

1. Overstocking of slow-moving SKUs leading to cash blockage
2. Stockouts of high-demand SKUs causing revenue loss
3. High manufacturing and logistics costs impacting margins
4. Supplier delays reducing service levels
5. Product defects increasing returns and rework costs

### Objective
Build a data-driven framework to:
- Identify high-risk SKUs
- Optimize reorder decisions
- Improve service levels
- Reduce operational costs

#### Importing Libraries

In [6]:
import pandas as pd
import numpy as np

#### Loading the Dataset

In [8]:
df = pd.read_csv("D:/Projects/Supply Chain Management Project/data/supply_chain_data.csv")
df.head()

,Product type,SKU,Price,Availability,Number of products sold,Revenue generated,Customer demographics,Stock levels,Lead times,Order quantities,...,Location,Lead time,Production volumes,Manufacturing lead time,Manufacturing costs,Inspection results,Defect rates,Transportation modes,Routes,Costs
0,haircare,SKU0,69.808006,55,802,8661.996792,Non-binary,58,7,96,...,Mumbai,29,215,29,46.279879,Pending,0.226410,Road,Route B,187.752075
1,skincare,SKU1,14.843523,95,736,7460.900065,Female,53,30,37,...,Mumbai,23,517,30,33.616769,Pending,4.854068,Road,Route B,503.065579
2,haircare,SKU2,11.319683,34,8,9577.749626,Unknown,1,10,88,...,Mumbai,12,971,27,30.688019,Pending,4.580593,Air,Route C,141.920282
3,skincare,SKU3,61.163343,68,83,7766.836426,Non-binary,23,13,59,...,Kolkata,24,937,18,35.624741,Fail,4.746649,Rail,Route A,254.776159
4,skincare,SKU4,4.805496,26,871,2686.505152,Non-binary,5,3,56,...,Delhi,5,414,3,92.065161,Fail,3.145580,Air,Route A,923.440632


## Data Quality Validation

Before analysis, we should ensure:
- No missing SKUs
- No negative stock values
- No zero-price products

In [12]:
df.info()
df.isnull().sum()
df.duplicated().sum()

# Business sanity checks
assert (df['Stock levels'] >= 0).all() # Checks all stocks level >=0 if any in negative returns completely as FALSE
assert (df['Price'] > 0).all() # Checks Price for the SKUs if any price = or < 0, returns ASSERTION ERROR and program STOPS

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Product type             100 non-null    object 
 1   SKU                      100 non-null    object 
 2   Price                    100 non-null    float64
 3   Availability             100 non-null    int64  
 4   Number of products sold  100 non-null    int64  
 5   Revenue generated        100 non-null    float64
 6   Customer demographics    100 non-null    object 
 7   Stock levels             100 non-null    int64  
 8   Lead times               100 non-null    int64  
 9   Order quantities         100 non-null    int64  
 10  Shipping times           100 non-null    int64  
 11  Shipping carriers        100 non-null    object 
 12  Shipping costs           100 non-null    float64
 13  Supplier name            100 non-null    object 
 14  Location                 10

## Key Assumptions

1. Number of products sold represents recent demand (snapshot-based).
2. Lead times are assumed stable for the analysis period.
3. Availability is used as a proxy for service level.
4. Cost components are assumed additive.
5. Demand variability is approximated using overall demand standard deviation.

These assumptions are necessary due to the absence of time-series data.

## Demand & Inventory Distribution Overview

This section provides high-level visibility into demand and stock patterns
before applying optimization logic.

In [16]:
df[['Number of products sold', 'Stock levels', 'Revenue generated']].describe()

,Number of products sold,Stock levels,Revenue generated
count,100.000000,100.000000,100.000000
mean,460.990000,47.770000,5776.048187
std,303.780074,31.369372,2732.841744
min,8.000000,0.000000,1061.618523
25%,184.250000,16.750000,2812.847151
50%,392.500000,47.500000,6006.352023
75%,704.250000,73.000000,8253.976921
max,996.000000,100.000000,9866.465458


## Key Business KPIs

Each KPI is selected to solve a specific business problem:

| KPI | Business Problem Solved |
|----|-------------------------|
| Demand | Understand customer pull |
| Inventory Turnover | Identify slow-moving stock |
| Stockout Risk | Prevent lost sales |
| Overstock Risk | Reduce cash blockage |
| Cost Efficiency | Protect margins |
| Quality Risk | Minimize returns |


### Inventory Turnover

**Why this KPI is being used:**
High inventory is not bad unless it does not sell.

**Business Problem it Solves:**
Identifies slow-moving or dead inventory that blocks working capital.


In [20]:
df['inventory_turnover'] = df['Number of products sold'] / (df['Stock levels'] + 1)

**Operational Actions:**
- **Low Turnover SKUs:**
  - Trigger discounting / bundling campaigns
  - Reduce future replenishment quantities
  - Evaluate product rationalization (discontinue if persistently low)
- **High Turnover SKUs:**
  - Increase replenishment frequency
  - Prioritize supplier capacity and allocation
  - Protect availability during peak demand

### Stockout Risk

**Why this KPI is being used:**
Revenue is lost when demand exceeds available stock.

**Business Problem it Solves:**
Identifies SKUs at immediate risk of losing sales and it enables priority replenishment.

In [24]:
daily_demand = df['Number of products sold'] / 30  # Assuming monthly data
df['stockout_risk'] = np.where(
    df['Stock levels'] < (daily_demand * df['Lead times']),  # Can't cover lead time demand
    1, 0
)

**Operational Actions:**
- Set **priority replenishment flags** for high-risk SKUs
- Introduce safety stock buffers for:
  - High-demand
  - Long lead-time products
- Re-route inventory between locations if feasible

### Overstock Risk

**Why this KPI is being used:**
Excess stock with low demand increases holding costs.

**Business Problem it Solves:**
Flags cash blockage and identify inventory that should be discounted, bundled, or discontinued.


In [28]:
df['overstock_risk'] = np.where(
    (df['Stock levels'] > df['Number of products sold']) &
    (df['inventory_turnover'] < 0.5),
    1, 0
)

**Operational Actions:**
- Freeze or reduce reordering
- Launch clearance or promotion strategies
- Reallocate inventory to higher-demand regions
- Review demand forecasting assumptions

### Cost Efficiency

**Why this KPI is being used:**
Revenue alone hides true profitability.

**Business Problem it Solves:**
Identifies SKUs that sell well but with low margins due to high costs.


In [32]:
df['total_cost'] = (
    df['Manufacturing costs'] +
    df['Shipping costs'] +
    df['Costs']
)

df['cost_efficiency'] = df['Revenue generated'] / (df['total_cost'] + 1)

**Operational Actions:**
- Identify SKUs with:
  - High sales but poor margins
- Renegotiate supplier pricing
- Optimize logistics or sourcing routes
- Revisit pricing strategies

### ABC Analysis (Revenue-Based)

**Why this KPI is being used:**
Not all SKUs deserve the same attention.

**Business Problem it Solves:**
Focuses planning efforts on revenue-critical products.


In [36]:
df = df.sort_values('Revenue generated', ascending=False)
df['cum_revenue_pct'] = df['Revenue generated'].cumsum() / df['Revenue generated'].sum()

df['ABC_category'] = np.where(
    df['cum_revenue_pct'] <= 0.7, 'A',
    np.where(df['cum_revenue_pct'] <= 0.9, 'B', 'C')
)

**Operational Actions:**
- **A-class SKUs:**
  - Highest service level targets
  - Zero tolerance for stockouts
- **B-class SKUs:**
  - Controlled inventory, demand-driven planning
- **C-class SKUs:**
  - Minimal safety stock
  - Candidates for discontinuation or made-to-order

### Inventory Optimization

**Why this KPI is being used:**
Different SKUs require different service levels based on business importance.

**Business Problem it Solves:** Safety Stock is used because, Lead times fluctuate, Demand is uncertain so it prevents Stockouts without Overstocking.

In [40]:
z_map = {'A': 2.05, 'B': 1.65, 'C': 1.28}
df['z_score'] = df['ABC_category'].map(z_map)

demand_cv = 0.3  # Assume 30% demand variability (industry standard)

df['safety_stock'] = (
    df['z_score'] *
    (df['Number of products sold'] / 30) *  # Daily demand
    demand_cv *  # Variability factor
    np.sqrt(df['Lead times'])
)

df['reorder_point'] = (
    (df['Number of products sold'] / 30) * df['Lead times']
) + df['safety_stock']

**Operational Actions:**
- Use EOQ to:
  - Reduce ordering + holding costs
- Use Safety Stock to:
  - Protect service levels under variability
- Adjust parameters when:
  - Lead times change
  - Demand volatility increases

### Supply Chain Risk Analysis

**Why this KPI is being used:**
Operational risks silently increase cost and reduce service levels.


In [44]:
df['quality_risk'] = np.where(df['Defect rates'] > 3, 1, 0)
df['supplier_delay_risk'] = np.where(df['Lead times'] > 20, 1, 0)

**Operational Actions:**
- Dual-source critical SKUs
- Negotiate flexible contracts
- Create contingency inventory buffers
- Monitor supplier reliability trends

## Demand-to-Stock Ratio

**Why this KPI is being used:**
Inventory turnover alone does not clearly show urgency.

**Business Problem it Solves:**
- Identifies SKUs where demand is disproportionately higher than stock
- Helps prioritize replenishment when budget or capacity is limited

In [48]:
df['demand_stock_ratio'] = df['Number of products sold'] / (df['Stock levels'] + 1)

**Operational Actions:**
1. **> 1** → High stockout urgency → Urgent replenishment
2. **~ 1** → Balanced → Maintain current policy
3. **< 0.5** → Overstock risk → Overstock mitigation actions

## Revenue at Risk

**Why this KPI is being used:**
Stockout flags alone do not quantify financial impact.

**Business Problem it Solves:**
- Converts operational risk into monetary terms
- Helps leadership prioritize SKUs based on revenue impact, not just volume

In [52]:
# Calculate potential lost revenue based on unmet demand
df['unmet_demand'] = np.maximum(
    df['Number of products sold'] - df['Stock levels'], 
    0
)
df['revenue_at_risk'] = df['unmet_demand'] * df['Price']

**Operational Actions:**
- Rank SKUs by **revenue exposure**
- Allocate limited inventory to highest-impact products
- Inform leadership decisions using ₹ impact, not volume

## Inventory Holding Risk Score

**Why this KPI is being used:**
Overstock flags are binary and lack severity.

**Business Problem it Solves:**
- Quantifies how dangerous excess inventory is
- Helps prioritize liquidation, discounts, or production cuts

**Interpretation:**
Higher score → Higher holding cost & cash blockage


In [56]:
df['holding_risk_score'] = (
    (df['Stock levels'] / (df['Number of products sold'] + 1)) *
    (1 / (df['inventory_turnover'] + 0.01))
)

**Operational Actions:**
- Reduce holding period for high-risk SKUs
- Improve demand forecasting accuracy
- Evaluate warehouse utilization

## Fulfillment Stress Index

**Why this KPI is needed:**
Demand spikes combined with long lead times amplify risk.

**Business Problem Solved:**
- Identifies SKUs where supplier delays will cause service failures
- Supports supplier negotiation and alternate sourcing decisions


In [60]:
df['fulfillment_stress_index'] = (
    df['Lead times'] *
    df['demand_stock_ratio']
)

**Operational Actions:**
- Renegotiate lead times
- Onboard alternate suppliers
- Improve inbound planning accuracy

## Margin Risk Indicator

**Why this KPI is needed:**
High sales do not guarantee profitability.

**Business Problem Solved:**
- Identifies SKUs destroying margins
- Supports pricing, sourcing, or discontinuation decisions


In [64]:
df['estimated_margin'] = df['Revenue generated'] - df['total_cost']

df['margin_risk_flag'] = np.where(
    (df['estimated_margin'] < 0) | (df['cost_efficiency'] < 1),
    1, 0
)

**Operational Actions:**
- Identify SKUs with:
  - High volume but low profitability
- Adjust pricing or sourcing
- Discontinue non-viable products

## Location-Level Service Risk

**Why this KPI is needed:**
Operational performance varies by geography.

**Business Problem Solved:**
- Identifies underperforming cities or warehouses
- Enables hyperlocal inventory optimization


In [68]:
location_service = (
    df.groupby('Location')
    .agg(
        total_skus=('SKU', 'count'),
        stockout_skus=('stockout_risk', 'sum'),
        avg_availability=('Availability', 'mean')
    )
    .reset_index()
)

location_service['location_stockout_rate'] = (
    location_service['stockout_skus'] /
    location_service['total_skus']
)

**Operational Actions:**
- Rebalance inventory across locations
- Improve hyperlocal demand forecasting
- Customize safety stock by location

## Composite Inventory Risk Score

**Why this KPI is needed:**
Decision-makers need a single prioritization metric.

**Business Problem Solved:**
- Enables SKU prioritization across multiple risk dimensions
- Supports executive-level decision-making


In [72]:
df['inventory_risk_score'] = (
    (df['stockout_risk'] * 0.4) +
    (df['overstock_risk'] * 0.2) +
    (df['quality_risk'] * 0.2) +
    (df['supplier_delay_risk'] * 0.2)
)

**Operational Actions:**
- Prioritize SKUs with highest composite risk
- Align cross-functional teams on mitigation
- Track improvements quarter-over-quarter

In [75]:
## Exporting Enriched Dataset with KPIs

# Select relevant columns for export (matching actual CSV structure)
kpi_columns = [
    # Original Data Columns
    'SKU', 'Product type', 'Price', 'Availability',
    'Number of products sold', 'Revenue generated', 
    'Customer demographics', 'Stock levels', 'Lead times', 
    'Order quantities', 'Shipping times', 'Shipping carriers', 
    'Shipping costs', 'Supplier name', 'Location', 
    'Lead time', 'Production volumes', 'Manufacturing lead time',
    'Manufacturing costs', 'Inspection results', 'Defect rates',
    'Transportation modes', 'Routes', 'Costs',
    
    # Calculated KPIs
    'inventory_turnover', 'stockout_risk', 'overstock_risk',
    'total_cost', 'cost_efficiency', 
    'cum_revenue_pct', 'ABC_category',
    'z_score', 'safety_stock', 'reorder_point',
    'quality_risk', 'supplier_delay_risk',
    'demand_stock_ratio', 'revenue_at_risk',
    'holding_risk_score', 'fulfillment_stress_index',
    'estimated_margin', 'margin_risk_flag',
    'inventory_risk_score'
]

In [77]:
# Create export dataframe
df_export = df[kpi_columns].copy()

# Sort by composite risk score (highest priority first)
df_export = df_export.sort_values('inventory_risk_score', ascending=False)

# Reset index for clean export
df_export = df_export.reset_index(drop=True)

# Export to CSV
output_path = "D:/Projects/Supply Chain Management Project/data/supply_chain_kpis_enriched.csv"
df_export.to_csv(output_path, index=False)

In [79]:
print(f"✓ Exported {len(df_export)} SKUs with {len(kpi_columns)} features")
print(f"✓ File saved as: {output_path}")

# Summary Statistics for Validation
print("\n" + "="*60)
print("KPI SUMMARY STATISTICS")
print("="*60)

print(f"\n Risk Distribution:")
print(f"   • High Risk SKUs (score > 0.5): {(df_export['inventory_risk_score'] > 0.5).sum()}")
print(f"   • Stockout Risk SKUs: {df_export['stockout_risk'].sum()}")
print(f"   • Overstock Risk SKUs: {df_export['overstock_risk'].sum()}")
print(f"   • Quality Risk SKUs: {df_export['quality_risk'].sum()}")
print(f"   • Supplier Delay Risk SKUs: {df_export['supplier_delay_risk'].sum()}")

print(f"\n Financial Impact:")
print(f"   • Total Revenue at Risk: ₹{df_export['revenue_at_risk'].sum():,.2f}")
print(f"   • SKUs with Negative Margins: {df_export['margin_risk_flag'].sum()}")
print(f"   • Average Cost Efficiency: {df_export['cost_efficiency'].mean():.2f}")

print(f"\n Inventory Health:")
print(f"   • Average Inventory Turnover: {df_export['inventory_turnover'].mean():.2f}")
print(f"   • Average Safety Stock: {df_export['safety_stock'].mean():.0f} units")
print(f"   • Total Stock Value: ₹{(df_export['Stock levels'] * df_export['Price']).sum():,.2f}")

# ABC Distribution
print("\n ABC Category Distribution:")
abc_dist = df_export['ABC_category'].value_counts().sort_index()
for category, count in abc_dist.items():
    pct = (count / len(df_export)) * 100
    print(f"   • Category {category}: {count} SKUs ({pct:.1f}%)")

# Product Type Analysis
print("\n Product Type Distribution:")
product_dist = df_export['Product type'].value_counts()
for prod_type, count in product_dist.items():
    print(f"   • {prod_type.title()}: {count} SKUs")

# Location-wise Analysis
print("\n Location-wise Stockout Rate:")
for _, row in location_service.sort_values('location_stockout_rate', ascending=False).iterrows():
    print(f"   • {row['Location']}: {row['location_stockout_rate']*100:.1f}% ({row['stockout_skus']}/{row['total_skus']} SKUs)")

# Export location-level analysis separately
location_output = "D:/Projects/Supply Chain Management Project/data/location_service_analysis.csv"
location_service.to_csv(location_output, index=False)
print(f"\n✓ Location analysis exported as: {location_output}")

# Create high-priority action list (Top 20 risky SKUs)
high_priority = df_export.head(20)[['SKU', 'Product type', 'ABC_category', 
                                      'stockout_risk', 'overstock_risk', 
                                      'revenue_at_risk', 'inventory_risk_score']]
priority_output = "D:/Projects/Supply Chain Management Project/data/high_priority_skus.csv"
high_priority.to_csv(priority_output, index=False)
print(f"✓ Top 20 high-priority SKUs exported as: {priority_output}")

print("\n" + "="*60)
print("EXPORT COMPLETE")
print("="*60)

✓ Exported 100 SKUs with 43 features
✓ File saved as: D:/Projects/Supply Chain Management Project/data/supply_chain_kpis_enriched.csv

KPI SUMMARY STATISTICS

 Risk Distribution:
   • High Risk SKUs (score > 0.5): 47
   • Stockout Risk SKUs: 81
   • Overstock Risk SKUs: 1
   • Quality Risk SKUs: 32
   • Supplier Delay Risk SKUs: 34

 Financial Impact:
   • Total Revenue at Risk: ₹2,044,099.66
   • SKUs with Negative Margins: 0
   • Average Cost Efficiency: 13.23

 Inventory Health:
   • Average Inventory Turnover: 24.47
   • Average Safety Stock: 30 units
   • Total Stock Value: ₹243,857.44

 ABC Category Distribution:
   • Category A: 49 SKUs (49.0%)
   • Category B: 24 SKUs (24.0%)
   • Category C: 27 SKUs (27.0%)

 Product Type Distribution:
   • Skincare: 40 SKUs
   • Haircare: 34 SKUs
   • Cosmetics: 26 SKUs

 Location-wise Stockout Rate:
   • Delhi: 93.3% (14/15 SKUs)
   • Chennai: 85.0% (17/20 SKUs)
   • Bangalore: 77.8% (14/18 SKUs)
   • Mumbai: 77.3% (17/22 SKUs)
   • Kolkata:

# Executive Summary & Business Impact

**Key Takeaways:**
- This analysis converts raw inventory data into **decision-grade intelligence**
- Risks are quantified across **revenue, cost, service, and operations**
- Enables proactive inventory planning instead of reactive firefighting

**Business Outcomes Enabled:**
- Reduced stockouts & overstock
- Improved service levels
- Lower working capital blockage
- Better executive visibility